# Web Scraping - Lab

## Introduction

Now that you've seen a more extensive example of developing a web scraping script, it's time to further practice and formalize that knowledge by writing functions to parse specific pieces of information from the web page and then synthesizing these into a larger loop that will iterate over successive web pages in order to build a complete dataset.

## Objectives

You will be able to:

* Navigate HTML documents using Beautiful Soup's children and sibling relations
* Select specific elements from HTML using Beautiful Soup
* Use regular expressions to extract items with a certain pattern within Beautiful Soup
* Determine the pagination scheme of a website and scrape multiple pages

## Lab Overview

This lab will build upon the previous lesson. In the end, you'll look to write a script that will iterate over all of the pages for the demo site and extract the title, price, star rating and availability of each book listed. Building up to that, you'll formalize the concepts from the lesson by writing functions that will extract a list of each of these features for each web page. You'll then combine these functions into the full script which will look something like this:  

```python
df = pd.DataFrame()
for i in range(2,51):
    url = "http://books.toscrape.com/catalogue/page-{}.html".format(i)
    soup = BeautifulSoup(html_page.content, 'html.parser')
    new_titles = retrieve_titles(soup)
    new_star_ratings = retrieve_ratings(soup)
    new_prices = retrieve_prices(soup)
    new_avails = retrieve_avails(soup)
    ...
 ```

In [1]:
# import necessary libraries
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re

In [2]:
# try pulling titles first from one page
# grab the soup from the first page and inspect
page = requests.get('http://books.toscrape.com/catalogue/category/books_1/index.html')
soup = BeautifulSoup(page.content, 'html.parser')

soup.prettify

<bound method Tag.prettify of 
<!DOCTYPE html>

<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-us"> <!--<![endif]-->
<head>
<title>
    Books | 
     Books to Scrape - Sandbox

</title>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="24th Jun 2016 09:29" name="created"/>
<meta content="
    
" name="description"/>
<meta content="width=device-width" name="viewport"/>
<meta content="NOARCHIVE,NOCACHE" name="robots"/>
<!-- Le HTML5 shim, for IE6-8 support of HTML elements -->
<!--[if lt IE 9]>
        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>
        <![endif]-->
<link href="../../../static/oscar/favicon.ico" rel="shortcut icon"/>
<link href="../../../static/oscar/css/style

In [3]:
# find the div marker of the container, using the class_ keyword argument to find the div at the top of the container
warning = soup.find('div', class_ ='alert alert-warning')
container = warning.nextSibling.nextSibling
container

<div>
<ol class="row">
<li class="col-xs-6 col-sm-4 col-md-3 col-lg-3">
<article class="product_pod">
<div class="image_container">
<a href="../../a-light-in-the-attic_1000/index.html"><img alt="A Light in the Attic" class="thumbnail" src="../../../media/cache/2c/da/2cdad67c44b002e7ead0cc35693c0e8b.jpg"/></a>
</div>
<p class="star-rating Three">
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
</p>
<h3><a href="../../a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a></h3>
<div class="product_price">
<p class="price_color">£51.77</p>
<p class="instock availability">
<i class="icon-ok"></i>
    
        In stock
    
</p>
<form>
<button class="btn btn-primary btn-block" data-loading-text="Adding..." type="submit">Add to basket</button>
</form>
</div>
</article>
</li>
<li class="col-xs-6 col-sm-4 col-md-3 col-lg-3">
<article class="product_pod">
<div class="image_contain

In [4]:
titles = container.find_all('h3') # this will strip out ALL the titles in the container into a list.
titles[4].find('a').attrs['title'] # but, for each member of the list, the ACTUAL title must be further stripped out

'Sapiens: A Brief History of Humankind'

## Retrieving Titles

To start, write a function that extracts the titles of the books on a given page. The input for the function should be the `soup` for the HTML of the page.

In [5]:
def retrieve_titles(soup):
    titles_on_page = []
    warning = soup.find('div', class_ ='alert alert-warning')
    container = warning.nextSibling.nextSibling
    titles = container.find_all('h3')
    for i in range(len(titles)):
        title = titles[i].find('a').attrs['title']
        titles_on_page.append(title)
    return titles_on_page
 

In [6]:
# test that the function works
#make the soup of a different page
testpage = requests.get('http://books.toscrape.com/catalogue/category/books/historical-fiction_4/index.html')
testsoup = BeautifulSoup(testpage.content, 'html.parser')
testlist = retrieve_titles(testsoup)
testlist

['Tipping the Velvet',
 'Forever and Forever: The Courtship of Henry Longfellow and Fanny Appleton',
 'A Flight of Arrows (The Pathfinders #2)',
 'The House by the Lake',
 'Mrs. Houdini',
 'The Marriage of Opposites',
 'Glory over Everything: Beyond The Kitchen House',
 'Love, Lies and Spies',
 'A Paris Apartment',
 'Lilac Girls',
 'The Constant Princess (The Tudor Court #1)',
 'The Invention of Wings',
 'World Without End (The Pillars of the Earth #2)',
 'The Passion of Dolssa',
 'Girl With a Pearl Earring',
 'Voyager (Outlander #3)',
 'The Red Tent',
 'The Last Painting of Sara de Vos',
 'The Guernsey Literary and Potato Peel Pie Society',
 'Girl in the Blue Coat']

In [7]:
regex = re.compile('star-rating (.*)')
(container.findAll('p', {'class': regex}))

[<p class="star-rating Three">
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 </p>, <p class="star-rating One">
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 </p>, <p class="star-rating One">
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 </p>, <p class="star-rating Four">
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 </p>, <p class="star-rating Five">
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 </p>, <p class="star-rating One">
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class

## Retrieve Ratings

Next, write a similar function to retrieve the star ratings on a given page. Again, the function should take in the `soup` from the given HTML page and return a list of the star ratings for the books. These star ratings should be formatted as integers.

In [8]:
def retrieve_ratings(soup):
    ratings_dict = {'One':1, 'Two':2, 'Three':3, 'Four':4, 'Five':5}
    star_ratings = [ ]
    regex = re.compile('star-rating (.*)')
    warning = soup.find('div', class_ ='alert alert-warning')
    container = warning.nextSibling.nextSibling
    ratings = container.findAll('p', {'class': regex})
    for i in ratings:
        book_rating = i.attrs['class'][-1] 
        star_ratings.append(book_rating)
    star_ratings = [ratings_dict[s] for s in star_ratings]
    return star_ratings

In [9]:
retrieve_ratings(soup)

[3, 1, 1, 4, 5, 1, 4, 3, 4, 1, 2, 4, 5, 5, 5, 3, 1, 1, 2, 2]

## Retrieve Prices

Now write a function to retrieve the prices on a given page. The function should take in the `soup` from the given page and return a list of prices formatted as floats.

In [10]:
p = container.findAll(class_='price_color')
p

[<p class="price_color">£51.77</p>,
 <p class="price_color">£53.74</p>,
 <p class="price_color">£50.10</p>,
 <p class="price_color">£47.82</p>,
 <p class="price_color">£54.23</p>,
 <p class="price_color">£22.65</p>,
 <p class="price_color">£33.34</p>,
 <p class="price_color">£17.93</p>,
 <p class="price_color">£22.60</p>,
 <p class="price_color">£52.15</p>,
 <p class="price_color">£13.99</p>,
 <p class="price_color">£20.66</p>,
 <p class="price_color">£17.46</p>,
 <p class="price_color">£52.29</p>,
 <p class="price_color">£35.02</p>,
 <p class="price_color">£57.25</p>,
 <p class="price_color">£23.88</p>,
 <p class="price_color">£37.59</p>,
 <p class="price_color">£51.33</p>,
 <p class="price_color">£45.17</p>]

In [11]:
p = container.findAll(class_='price_color')[3].text #note the .text method to get the actual figure.
float(p[1:]) #strip the pound sign in p and turn the remainder into a float.

47.82

In [12]:
def retrieve_prices(soup):
    pricelist = []
    warning = soup.find('div', class_ ='alert alert-warning')
    container = warning.nextSibling.nextSibling
    prices = container.findAll(class_='price_color')
    for j in range(len(prices)):
        one_price = prices[j].text
        float_price = float(one_price[1:])
        pricelist.append(float_price)
    return pricelist
    #Your code here

In [13]:
retrieve_prices(soup)

[51.77,
 53.74,
 50.1,
 47.82,
 54.23,
 22.65,
 33.34,
 17.93,
 22.6,
 52.15,
 13.99,
 20.66,
 17.46,
 52.29,
 35.02,
 57.25,
 23.88,
 37.59,
 51.33,
 45.17]

## Retrieve Availability

Write a function to retrieve whether each book is available or not. The function should take in the `soup` from a given html page and return a list of the availability for each book.

In [14]:
def retrieve_availabilities(soup):
    list_avails = [ ]
    warning = soup.find('div', class_ ='alert alert-warning')
    container = warning.nextSibling.nextSibling
    avails = container.findAll('p', class_="instock availability")
    for i in range(len(avails)):
        indiv_avail = avails[i].text.strip()
        list_avails.append(indiv_avail)
    return list_avails
    #Your code here

## Create a Script to Retrieve All the Books From All 50 Pages

Finally, write a script to retrieve all of the information from all 50 pages of the books.toscrape.com website. 

In [15]:
def retrieve_all(soup):
    df = pd.DataFrame([retrieve_titles(soup), retrieve_ratings(soup), retrieve_prices(soup), retrieve_availabilities(soup)]).transpose()
    df.columns = ['Title', 'Star_Ratings', 'Prices', 'Availability']
    return df

df = retrieve_all(soup)

for i in range(2,51):
    url = 'http://books.toscrape.com/catalogue/page-{}.html'.format(i)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    each_page = retrieve_all(soup)
    to_concat = [df, each_page]
    df = pd.concat(to_concat)


In [16]:
df

,Title,Star_Ratings,Prices,Availability
0,A Light in the Attic,3,51.77,In stock
1,Tipping the Velvet,1,53.74,In stock
2,Soumission,1,50.1,In stock
3,Sharp Objects,4,47.82,In stock
4,Sapiens: A Brief History of Humankind,5,54.23,In stock
...,...,...,...,...
15,Alice in Wonderland (Alice's Adventures in Won...,1,55.53,In stock
16,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)",4,57.06,In stock
17,A Spy's Devotion (The Regency Spies of London #1),5,16.97,In stock
18,1st to Die (Women's Murder Club #1),1,53.98,In stock


In [23]:
g = container.find('next')
type(g)

NoneType

## Level-Up: Write a new version of the script you just wrote. 

If you used URL hacking to generate each successive page URL, instead write a function that retrieves the link from the `"next"` button at the bottom of the page. Conversely, if you already used this approach above, use URL-hacking (arguably the easier of the two methods in this case).

In [ ]:
#Your code here

## Summary

Well done! You just completed your first full web scraping project! You're ready to start harnessing the power of the web!